In [1]:
from traversal import viewtree

In [2]:
pattern = '{(\w+)}'

In [3]:
from parser import parser

In [4]:
root = parser.parse('^author (.*)')

In [5]:
type(root)

parser.Node

In [6]:
root

Expression([Anchor([], ^), Expression([String([String([String([String([String([String([String([None], Whitespace([],  ))], Letter([], r))], Letter([], o))], Letter([], h))], Letter([], t))], Letter([], u))], Letter([], a)), Expression([Group([Expression([String([], .*), None], None)], capture), None], None)], None)], None)

In [7]:
def preorder(root):
    if not root: return
    res = []
    stack = []
    level = 1
    stack.append((root, level))

    while len(stack) > 0:
        node, level = stack.pop()
        res.append((node.type, node.value, level))
        print(' '*level + f'{node.type}({node.value})')

        for child in node.children[::-1]:
            if child:
                stack.append((child,level+1))
    return res

In [8]:
def postorder(root):
    stack = []
    res = []
    level = 1
    stack.append((root, level))
    
    while len(stack) > 0:
        node, level = stack.pop()
        res.insert(0, (node.type, node.value, level))
        for child in node.children:
            if child:
                stack.append((child, level+1))
                
    return res  

In [54]:
traversal = postorder(root)
traversal

[('Anchor', '^', 2),
 ('String', Whitespace([],  ), 9),
 ('String', Letter([], r), 8),
 ('String', Letter([], o), 7),
 ('String', Letter([], h), 6),
 ('String', Letter([], t), 5),
 ('String', Letter([], u), 4),
 ('String', Letter([], a), 3),
 ('String', '.*', 6),
 ('Expression', None, 5),
 ('Group', 'capture', 4),
 ('Expression', None, 3),
 ('Expression', None, 2),
 ('Expression', None, 1)]

In [56]:
build_expression(traversal)

[[('Anchor', '^', 2)],
 [('String', Whitespace([],  ), 9),
  ('String', Letter([], r), 8),
  ('String', Letter([], o), 7),
  ('String', Letter([], h), 6),
  ('String', Letter([], t), 5),
  ('String', Letter([], u), 4),
  ('String', Letter([], a), 3)],
 [('String', '.*', 6),
  ('Expression', None, 5),
  ('Group', 'capture', 4),
  ('Expression', None, 3),
  ('Expression', None, 2),
  ('Expression', None, 1)]]

In [9]:
from collections import defaultdict

In [52]:
def build_expression(traversal):
    paths = []
    path = []
    i = 0
    
    while True:
        j = i + 1
        if j == len(traversal):
            path.append(traversal[i])
            paths.append(path)
            return paths
        
        if traversal[j][2] > traversal[i][2]:
            path.append(traversal[i])
            paths.append(path)
            path = []
        elif traversal[j][2] < traversal[i][2]:
            path.append(traversal[i])
        
        i += 1
    return paths

In [46]:
def parse_expression(expression):
    subtrees = defaultdict(list)
    
    for path in expression:
        # Check for leaf nodes
        if len(path) == 1: 
            # Anchor tag string value
            if path[0][0] == 'Anchor':
                if path[0][1] == '$':
                    subtrees[path[0][2]].append('end of line')
                elif path[0][1] == '^':
                    subtrees[path[0][2]].append('start of line')
        else:
            types = list(set(node[0] for node in path))
            # If all nodes on a path are the same type
            if len(types) == 1:
                # Concatenate string nodes into single string
                if types[0] == 'String':
                    string = ''
                    for node in path[::-1]:
                        string += node[1].value
                    subtrees[path[-1][2]].append(f'string "{string}"')
            else:
                result = ''
                for curr, next in zip(path, path[1:]):
                    
                    
    return subtrees

In [49]:
traversal = postorder(parser.parse('^author (.*)'))
print(traversal)
preorder(parser.parse('^author (.*)'))
expression = build_expression(traversal)
expression

[('Anchor', '^', 2), ('String', Whitespace([],  ), 9), ('String', Letter([], r), 8), ('String', Letter([], o), 7), ('String', Letter([], h), 6), ('String', Letter([], t), 5), ('String', Letter([], u), 4), ('String', Letter([], a), 3), ('String', '.*', 6), ('Expression', None, 5), ('Group', 'capture', 4), ('Expression', None, 3), ('Expression', None, 2), ('Expression', None, 1)]
 Expression(None)
  Anchor(^)
  Expression(None)
   String(Letter([], a))
    String(Letter([], u))
     String(Letter([], t))
      String(Letter([], h))
       String(Letter([], o))
        String(Letter([], r))
         String(Whitespace([],  ))
   Expression(None)
    Group(capture)
     Expression(None)
      String(.*)


[[('Anchor', '^', 2)],
 [('String', Whitespace([],  ), 9),
  ('String', Letter([], r), 8),
  ('String', Letter([], o), 7),
  ('String', Letter([], h), 6),
  ('String', Letter([], t), 5),
  ('String', Letter([], u), 4),
  ('String', Letter([], a), 3)],
 [('String', '.*', 6),
  ('Expression', None, 5),
  ('Group', 'capture', 4),
  ('Expression', None, 3),
  ('Expression', None, 2),
  ('Expression', None, 1)]]

In [48]:
parse_expression(expression)

defaultdict(list,
            {2: ['start of line'],
             3: ['string "author "', ('String', '.*', 6)]})

In [14]:
traversal = postorder(parser.parse('{(\w+)}'))
expression = build_expression(traversal)
expression

[[('Word', '\\w+', 6),
  ('Expression', None, 5),
  ('Group', 'capture', 4),
  ('Expression', None, 3),
  ('Closure', 'curly brace', 2),
  ('Expression', None, 1)]]

In [57]:
traversal = postorder(parser.parse('.*\.(a|so|dylib)$'))
expression = build_expression(traversal)
expression

[[('String', '.*', 2)],
 [('String', Punctuation([], .), 3)],
 [('String', Letter([], a), 6)],
 [('String', Letter([], o), 9), ('String', Letter([], s), 8)],
 [('String', Letter([], b), 14),
  ('String', Letter([], i), 13),
  ('String', Letter([], l), 12),
  ('String', Letter([], y), 11),
  ('String', Letter([], d), 10),
  ('Expression', None, 9),
  ('Alternation', 'or', 8),
  ('Expression', None, 7),
  ('Alternation', 'or', 6),
  ('Expression', None, 5),
  ('Group', 'capture', 4)],
 [('Anchor', '$', 5),
  ('Expression', None, 4),
  ('Expression', None, 3),
  ('Expression', None, 2),
  ('Expression', None, 1)]]